In [1]:
import os

db_dir = os.environ["DATA"] + "PatImgXAI_data/db1.0.0/"
os.makedirs(db_dir, exist_ok=True)

test_datasets_sizes=1000
valid_datasets_sizes=1000
full_datasets_pos_samples_nb=5000
full_datasets_neg_samples_nb=5000

In [2]:
from xaipatimg.datagen.dbimg import load_db

db = load_db(db_dir)

In [ ]:
# Number of images generated
NBGEN = 2000000
NBGEN = 10

# Grid division of each image
X_DIVISIONS = 6
Y_DIVISIONS = 6

# Size of the images in pixels
img_size = (700, 700)

# Probability to generate a geometrical shape at each position in the grid
SHAPE_PROB = 0.5

# Define available shapes
SHAPES = ['circle', 'square', 'triangle']
COLORS  = ["#F86C62", "#7AB0CD", "#F4D67B", "#87C09C"]

In [ ]:
import numpy as np
from xaipatimg.datagen.dbimg import generate_uuid
import os

to_generate = NBGEN
unique_content_generated = {}
duplicate_count = 0
while to_generate > 0:
    content = []
    for i in range(X_DIVISIONS):
        for j in range(Y_DIVISIONS):
            if np.random.random() < SHAPE_PROB:
                content.append({
                    "shape": np.random.choice(SHAPES),
                    "pos": (i, j),
                    "color": np.random.choice(COLORS)
                })

    if str(content) in unique_content_generated:
        duplicate_count += 1
        continue

    imgid = generate_uuid()
    db[imgid] = {
        "path": os.path.join("img", imgid + ".png"),
        "division" : (X_DIVISIONS, Y_DIVISIONS),
        "size": img_size,
        "content": content
    }

    unique_content_generated[str(content)] = True
    to_generate -= 1

print("unique generated in DB : " + str(len(db)))
print("duplicates avoided : " + str(duplicate_count))

In [ ]:
import tqdm

content_dict = {}
nb_duplicates = 0

for k, v in tqdm.tqdm(db.items()):
    if str(v["content"]) in content_dict:
        nb_duplicates += 1
    else:
        content_dict[str(v["content"])] = True

print(nb_duplicates)

In [ ]:
from xaipatimg.datagen.genimg import gen_img_and_save_db
# gen_img_and_save_db(db, db_dir, overwrite=False, n_jobs=20)

In [3]:
import numpy as np

def extract_rows_with_only_circles(img_content):
    circles_counter = np.zeros(6,)
    non_circles_counter = np.zeros(6,)

    for c in img_content:
        if c["shape"] == "circle":
            circles_counter[c["pos"][1]] += 1
        else:
            non_circles_counter[c["pos"][1]] += 1

    return np.logical_and(circles_counter >= 1, non_circles_counter == 0)

def exist_row_with_only_circles(img_content):
    return np.sum(extract_rows_with_only_circles(img_content)) >= 1

In [4]:
from xaipatimg.datagen.gendataset import create_dataset_based_on_rule
import os
csv_name_train = "rowcircles_train.csv"
csv_name_test = "rowcircles_test.csv"
csv_name_valid = "rowcircles_valid.csv"
sample_img_path = os.path.join(db_dir, "datasets", "rowcircles_train")
create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_test, csv_name_valid,
                             test_size=test_datasets_sizes, valid_size=valid_datasets_sizes,
                             dataset_pos_samples_nb=full_datasets_pos_samples_nb, dataset_neg_samples_nb=full_datasets_neg_samples_nb,
                             rule_fun=exist_row_with_only_circles)

100%|██████████| 2000000/2000000 [00:39<00:00, 51008.68it/s]


Total number of positive instances found in database : 724103
Total number of negative instances found in database : 1275897


In [5]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, sample_img_path, 1000, 1000)

8001it [00:19, 407.00it/s] 


In [6]:
"""
Create a dataset where If row 2 contains at least one red symbol, Rule 1 applies; otherwise, Rule 2 applies:
Rule 1: The number of triangles is greater than or equal to the number of blue symbols.
Rule 2: The number of green symbols is greater than or equal to the number of squares.
"""
def red_in_row_2_rule(img_content):
    red_in_row_2 = False
    #find if row 2 contains any red symbols
    for c in img_content:
        if c['pos'][1] == 1 and c['color'] == "#F86C62":
            red_in_row_2 = True
            break

    if red_in_row_2:
        # Rule 1
        nb_triangles = 0
        nb_blue = 0
        for c in img_content:
            if c['shape'] == 'triangle':
                nb_triangles += 1
            if c['color'] == "#7AB0CD":
                nb_blue += 1
        return nb_triangles >= nb_blue
    else:
        # Rule 2
        nb_green = 0
        nb_squares = 0
        for c in img_content:
            if c['color'] == "#87C09C":
                nb_green += 1
            if c['shape'] == 'square':
                nb_squares += 1
        return nb_green >= nb_squares

csv_name_train = "red_in_row_2_train.csv"
csv_name_test = "red_in_row_2_test.csv"
csv_name_valid = "red_in_row_2_valid.csv"
sample_img_path = os.path.join(db_dir, "datasets", "red_in_row_2_train")

create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_test, csv_name_valid,
                             test_size=test_datasets_sizes, valid_size=valid_datasets_sizes,
                             dataset_pos_samples_nb=full_datasets_pos_samples_nb, dataset_neg_samples_nb=full_datasets_neg_samples_nb,
                             rule_fun=exist_row_with_only_circles)


100%|██████████| 2000000/2000000 [00:39<00:00, 50799.50it/s]


Total number of positive instances found in database : 724103
Total number of negative instances found in database : 1275897


In [7]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, sample_img_path, 1000, 1000)

8001it [00:00, 35479.15it/s]
